# 🔍 Building an AI-Powered Web Search Agent with OpenAI and Tavily 🚀

Hey there! Welcome to this exciting guide where we'll create something awesome - a smart search agent that combines the power of OpenAI's language models with Tavily's search capabilities! 🌟 

## 🎯 What We'll Build

We're going to create a super cool search agent that can:
1. 🌐 Search the web in real-time for accurate information
2. 🧠 Use OpenAI's powerful GPT models to understand and process search results
3. ⚡ Provide contextual and up-to-date responses to queries

## ✅ Prerequisites

Before we jump in, make sure you have these things ready:
- 🔑 An OpenAI API key
- 🎯 A Tavily API key (get one at tavily.com)

## 🎮 Part 1: Setting Up Our Environment

First things first - let's get our tools ready! We'll need to install the Tavily Python package to interact with their search API:


In [2]:
# Install necessary libraries
!pip install streamlit openai requests python-dotenv

In [3]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


In [4]:
if TAVILY_API_KEY:
    print("✅ Tavily API Key loaded successfully.")
else:
    print("❌ Error: Tavily API Key not found. Check your .env file.")


✅ Tavily API Key loaded successfully.



## 🛠️ Part 2: Building Our Search Tools

Let's create the foundation of our search agent! We'll define a set of tools that our AI can use to search the web:

In [5]:
import json
from openai import OpenAI
from tavily import TavilyClient
import pprint

In [6]:
# Initialize Tavily
tavily = TavilyClient(api_key=TAVILY_API_KEY)

# Search query Ombu Urban Lab
import requests

def web_search(city, topic, timeframe, num_results=5):
    query = f"{topic} urban planning report OR government policy study OR academic paper in {city} during {timeframe}"
    url = "https://api.tavily.com/search"
    headers = {"Authorization": f"Bearer {TAVILY_API_KEY}"}
    payload = {"query": query, "num_results": num_results}

    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json().get("results", [])
    else:
        print("Error:", response.text)
        return []


In [7]:
web_search("Tokyo", "Green Areas", "2020-2025", num_results=5)

[{'title': 'PDF',
  'url': 'https://phpmyadmin.cloud1.glc.org/Download_PDFS/virtual-library/U000002/2_Tokyo_S_Urban_Growth_Urban_Form_And_Sustainability.pdf',
  'content': "Tokyo, urban growth, urban form, sustainability, megacity, density, transportation, energy, waste management, environmental impact, urban planning, green spaces, resilience, smart city, innovation. Tokyo, one of the world's largest and most densely populated cities, has experienced phenomenal growth over the past century. This growth has",
  'score': 0.32611096,
  'raw_content': None},
 {'title': 'Urban form and its impacts on air pollution and access to green space ...',
  'url': 'https://pmc.ncbi.nlm.nih.gov/articles/PMC9876236/',
  'content': 'Access to green spaces in urban areas has been recognized as an essential component of the urban environment . Green spaces in cities mostly include semi-natural vegetation cover, such as street trees, lawns, parks, gardens, forests, green roofs . Therefore, this study aims

In [8]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for urban planning information, reports, studies, or academic papers.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query including topic, city, and timeframe"}
                },
                "required": ["query"]
            },
        },
    },
]



## 🎓 Part 3: Creating Our AI Agent

Now comes the exciting part! Let's create our AI agent that can understand questions and use our search tools to find answers:


In [14]:
from datetime import datetime

messages = [
    {
        "role": "system",
        "content": f"""
        You are a helpful Urban Research Assistant specializing in mobility, urban planning, green spaces, and public policies.
        
        Your mission is to:
        1. Search the web for serious sources like urban reports, policy documents, and academic research papers.
        2. List the best findings together with their links and bullet points showing their key insights.
            
        Make as many tool calls as needed to find good-quality sources before responding.
        Focus on reliable information, avoid blogs, tourism websites, and commercial content.
        
        Current date: {datetime.now().strftime('%Y-%m-%d')}
        """
    },
    {
        "role": "user",
        "content": "Find urban planning trends related to shared mobility in Utrecht for 2020-2025."
    }
]


In [15]:
def invoke_model(messages):
    # Initialize the OpenAI client
    client = OpenAI()

    # Make a ChatGPT API call with tool calling
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message.content

In [16]:
# Initialize the OpenAI client
client = OpenAI()

# Make a ChatGPT API call with tool calling
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    tools=TOOLS,
    messages=messages
)

response = completion.choices[0].message
pprint.pprint(response.tool_calls)

# Parse the response to get the tool call arguments
if response.tool_calls:
    # Process each tool call
    for tool_call in response.tool_calls:
        # Get the tool call arguments
        tool_call_arguments = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "search_web":
            print("Searching for", tool_call_arguments)
            search_results = search_web(tool_call_arguments["query"])
            messages.append({"role": "assistant", "content": f"{tool_call_arguments["query"]}: {search_results}"})
    print(invoke_model(messages))

else:
    # If there are no tool calls, return the response content
    print(response.content)

[ChatCompletionMessageToolCall(id='call_1thO3rTc8QFWpslAAzcOe0oj', function=Function(arguments='{"query":"Utrecht urban planning trends shared mobility 2020-2025"}', name='web_search'), type='function')]
I conducted a thorough search for recent trends in urban planning related to shared mobility specifically in Utrecht for the years 2020-2025. Below are key findings from academic reports, official urban planning documents, and research studies.

### Key Findings on Urban Planning Trends Related to Shared Mobility in Utrecht (2020-2025)

1. **Utrecht's Integrated Mobility Strategy**
   - **Source**: [Utrecht Mobility Strategy 2020-2025](https://www.utrecht.nl/fileadmin/uploads/documenten/algemeen/Mobiliteitsstrategie_Utrecht_2020-2025.pdf)
   - **Key Insights**:
     - Focus on integrating public transport and shared mobility services to enhance accessibility.
     - Emphasis on reducing car dependency through improved cycling infrastructure and public transport options.
     - Promotio

In [12]:
# create a function call the OpenAi API 

def agent(messages):

    # Initialize the OpenAI client
    client = OpenAI()

    # Make a ChatGPT API call with tool calling
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        tools=TOOLS, # here we pass the tools to the LLM
        messages=messages
    )

    # Get the response from the LLM
    response = completion.choices[0].message

    # Parse the response to get the tool call arguments
    if response.tool_calls:
        # Process each tool call
        for tool_call in response.tool_calls:
            # Get the tool call arguments
            tool_call_arguments = json.loads(tool_call.function.arguments)
            if tool_call.function.name == "save_memory":
                return save_memory(tool_call_arguments["memory"])
            elif tool_call.function.name == "web_search":
                search_results = search_web(tool_call_arguments["query"])
                messages.append({"role": "assistant", "content": f"Here are the search results: {search_results}"})
                return invoke_model(messages)
    else:
        # If there are no tool calls, return the response content
        return response.content

In [20]:
# create a function to help the user to create a hypothesis for a spatial analysis by prompting the user with questions and display the hypothesis in a bullet point format 
def create_hypothesis(messages):
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return completion.choices[0].message.content

In [21]:
create_hypothesis(messages)

'I found several credible sources discussing urban planning trends related to shared mobility in Utrecht from 2020 to 2025. Below are the key findings along with links to the reports and articles.\n\n1. **Utrecht City Mobility Plan 2020-2025**\n   - **Source**: Municipality of Utrecht\n   - **Link**: [Utrecht City Mobility Plan](https://www.utrecht.nl/bestuur-en-organisatie/over-de-gemeente/mobiliteit-van-utrecht/mobiliteitsplan-2020-2025/)\n   - **Key Insights**:\n     - The city aims to achieve more sustainable and efficient mobility solutions through shared mobility services.\n     - Emphasis on integrating public transport with shared transport modes like bikes, scooters, and cars.\n     - Plans for the expansion of shared bicycle schemes, with an emphasis on accessibility and sustainability.\n     - Introduction of digital platforms for better coordination of shared mobility services.\n\n2. **Shared Mobility and Urban Sustainability**\n   - **Source**: Academic Journal of Urban Pl

In [13]:
messages

[{'role': 'system',
  'content': '\n        You are a helpful Urban Research Assistant specializing in mobility, urban planning, green spaces, and public policies.\n        \n        Your mission is to:\n        1. Search the web for serious sources like urban reports, policy documents, and academic research papers.\n        2. List the best findings together with their links\n        2. Summarize the main points clearly in bullet form.\n        3. Prompt the user questions in order to co-create hypothesis for a spatial analysis\n        \n        Make as many tool calls as needed to find good-quality sources before responding.\n        Focus on reliable information, avoid blogs, tourism websites, and commercial content.\n        \n        Current date: 2025-04-28\n        '},
 {'role': 'user',
  'content': 'Find urban planning trends related to shared mobility in Utrecht for 2020-2025.'}]